In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Detecting “Pump & Dump Schemes” on Cryptocurrency Market Using An Improved Apriori Algorithm

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8705858 Main paper for this notebook

Data from https://www.kaggle.com/xblock/mtgox-leaked-transaction - Provided column types below


 ### Columns: 
		1.    Source : The user who sell bitcoins
		2.    Target : The user who buy bitcoins
		3.  Trade_Id : The ID of present trasaction
		4.  Bitcoins : Number of bitcoins involved in the current transaction
		5.     Money : Dollars spent buying bitcoin
		6. Money_rate : Price per bitcoin
		7.      Date : Date of transaction
		8.     label : There are three types of users: 0 for normal price transactions, 1 for abnormally high price transactions, and 2 for abnormally low price transactions. 

In [45]:
#define data types - converting IDs to str, otherwise they cause issues with pd.Series.value_counts()
dtypes = {"Source":str,
          "Target":str,
          "Trade_id":str,
          "Bitcoins":float,
          "Money":float,
          "Money_Rate":float,
          "Date":object,
          "label":int}
df = pd.read_csv("./data/complete_edge_v2.csv", dtype=dtypes, parse_dates=[6])

In [46]:
df.head()

,Source,Target,Trade_Id,Bitcoins,Money,Money_Rate,Date,label
0,895,3931,35372,23.020,18.061,0.784579,2011-04-01 00:28:54,0
1,895,722,35373,10.000,7.800,0.780000,2011-04-01 00:28:54,0
2,895,3605,35374,35.000,27.300,0.780000,2011-04-01 00:28:54,0
3,895,3966,35375,10.600,8.246,0.777925,2011-04-01 00:28:54,0
4,895,3353,35376,176.214,136.920,0.777010,2011-04-01 00:28:54,0


In [47]:
df.dtypes

Source                object
Target                object
Trade_Id               int64
Bitcoins             float64
Money                float64
Money_Rate           float64
Date          datetime64[ns]
label                  int64
dtype: object

Some data cleaning is done in the paper, but has already been done for us by Kaggle. The paper also treats multiple adjacent transaction records with the same buyer or seller as a single buy or sell (?). The paper also removes the records of all users who have 10 times more than the average number of transactions, as well "Markus" and "Willy", two known price manipulating bots. The Markus bot operated under a single userID, while the Willy bot used 49 different userIDs. I couldn't find what userIDs those were, so we can remove them later, if they aren't removed by the 10*mean threshold 

In [53]:
#Take the union of source and target, and count unique values
#Need to convert to strings because this data is werid
#
txn_counts = pd.concat([df["Source"], df["Target"]],ignore_index=True).value_counts()

In [55]:
txn_mean = txn_counts.mean() 
display(txn_mean) #Average number of transactions
txn_thresh = txn_mean*10 
display(txn_thresh) #Remove users with more transactions than the threshold

113.54024953285906

1135.4024953285907

In [64]:
(txn_counts>txn_thresh) #give me only the indexes that are true, then delete transactions in the main dataframe that contain these user_ids for source or target

231        True
THK        True
89169      True
1          True
115248     True
          ...  
380373    False
361920    False
253799    False
64163     False
357546    False
Length: 119343, dtype: bool

In [ ]:
#"improved apriori algo"
def apriori(t_matrix, mincnt, span):
    '''
    Inputs: 
    t_matrix - matrix which records the buy (or sell) transactions, where x is userid, and y is a time series, and x,y=1 if the user has bought at the timestamp
    mincnt - support threshold
    span - timespan - actions are considered simultanious if they occur within [t-span+1,t+span]
    
    Outputs:
    F: buying groups
    supports: supports for the buying groups...
    '''
    ###Part 1
    #create all k groups (F_k) and their supports (s_k)
    #All users
    C=rownames(t_matrix)
    #All users whose support exceeds the threshold
    F=[]
    #supports for users who exceed the threshold
    supports=[]
    for k in range(len(F)):
        #calculate support
        C[k]=#apriori-gen(F[k-1])
        F[k]=C[k]>minct#and add users who are above the support threshold (fix this line!)
        supports[k]=F[k].support #fix this line
    ###part 2
    def count(c):
        '''
        Calculates support for a given n-group with n >= 2
        c is a list of users, and ts are buy timestamps for that user
        '''
        #for each timestamp in each set of timestamps,
            #calculate the beginning of the interval: ti-span+1
            #calculate the end of the interval: ti+span
            if i == 1:
                if: #if during the interval, all users in this group bought, mark this as a group buying action
            else:
                if ti_begin < t(i-1)end #yadda... increment and continue
            return common_times
    return (F, supports)
    

This algorithm returns groups of users and the support for them having simultaneous buy transactions. In the "real world" where user and transaction IDs are not available (until they are leaked I guess), we can only look at price and time. In order to compare to the moving average technique from our paper, we need to modify this algorithm to return the timestamps where high amounts of users are purchasing/selling simultaneously, and get HLOC information about the coin during the timespan of the data leak.